<a href="https://colab.research.google.com/github/taiaraujo/EnemGenius/blob/main/EnemGenius_Prototipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# "EnemGenius_Prototipo.ipynb" 🚀📚

E aí, pessoal! Tudo tranquilo? 😎 Vamos desvendar juntos esse código incrível que te ajuda a arrasar no ENEM! 🤩 Preparei uma documentação supercompleta e fácil de entender, cheia de emojis e detalhes pra você não perder nada! 😉

## 1. Preparando o Terreno 🛠️

### 1.1. Instalando os Pacotes Necessários 📦




In [ ]:
# Instalação dos Pacotes necesários
!pip -q install google-genai
!pip -q install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

### 1.2. Importando as Bibliotecas 📚

In [ ]:
# Importação das Bibliotecas

import os
from google.colab import userdata

from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, FunctionTool
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import json


warnings.filterwarnings("ignore")

### 1.3 Configurando o Projeto ⚙️

In [ ]:
# Configurações do Projeto

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

## 2. As Funções Mágicas 🧙‍♀️

### 2.1. `call_agent()`: Chamando os Professores Virtuais 📞

- O que isso faz? 🤩

  - Essa função é tipo um "telefonista" que conecta você com os professores virtuais (os agentes de IA).
  - Ela pega o agente (o professor) e a sua pergunta (a mensagem).
  - Cria uma "conversa" (sessão) com o agente.
  - Envia a sua pergunta para o agente.
  - Espera a resposta do agente.
  - E te entrega a resposta final! 🎁

- Por que isso é importante? 🤓
  - É a função que permite que você converse com os professores virtuais e receba as respostas deles! 💬


### 2.2. `to_markdown():` Deixando o Texto Bonitão ✨

- O que isso faz? 🧐
  - Essa função é tipo um "maquiador" que deixa o texto mais bonito e fácil de ler no Colab.
  - Ela formata as listas (trocando os "•" por "*").
  - E coloca umas "margens" no texto (com o "> ") pra ficar mais organizado.

- Por que isso é importante? 🤩
  - Ajuda a gente a ler as respostas dos professores virtuais de forma mais agradável! 😉

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## 3. Os Professores Virtuais 🧑‍🏫

### 3.1. As "Instruções Base" 📜

- O que isso faz? 🤯
  - Essas são as "regras de ouro" que todos os professores virtuais devem seguir!
  - `INSTRUCTION_BASE`: Diz como o professor deve se comportar (ser experiente, claro, didático, etc.). 🤓
  - `RESPONSE_STRUCTURE`: Diz como a resposta do professor deve ser organizada (introdução, explicação, dicas, etc.). 📝

- Por que isso é importante? 🧐
  - Garante que todos os professores virtuais sejam consistentes e te deem a melhor ajuda possível! 🤩

In [ ]:
INSTRUCTION_BASE = '''
Você é um professor do ensino médio com vasta experiência na sua área.
Seu objetivo é ajudar alunos na preparação para o ENEM, fornecendo explicações claras, exemplos práticos e dicas relevantes.
Você deve utilizar a ferramenta de busca do Google (google_search) para encontrar informações atualizadas,
questões de provas anteriores do ENEM e materiais de estudo confiáveis sobre o tema solicitado pelo aluno.

Características importantes:
- Domínio Profundo: Demonstre conhecimento abrangente e atualizado sobre o tema.
- Clareza e Didática: Explique conceitos de forma clara, utilizando analogias e exemplos do cotidiano para facilitar a compreensão.
- Linguagem Acessível: Adapte a linguagem ao nível do aluno, evitando jargões excessivos, mas sem simplificar demais o conteúdo.
- Interatividade: Incentive a participação do aluno, fazendo perguntas, propondo exercícios e estimulando a reflexão crítica.
- Motivação: Transmita entusiasmo pelo tema, mostrando sua relevância.
- Foco no ENEM: Direcione o conteúdo para as competências e habilidades exigidas na prova, com dicas de estudo e resolução de questões.
'''

RESPONSE_STRUCTURE = '''
Estrutura da Resposta:
1. Introdução: Apresente o tema de forma geral, despertando o interesse do aluno.
2. Explanação Detalhada: Explique os conceitos-chave, utilizando exemplos e analogias.
3. Aplicações Práticas: Mostre como o tema se aplica em situações reais e no contexto do ENEM.
4. Questões e Exercícios: Envie links para questões de provas anteriores do enem já resolvidos.
5. Dicas e Macetes: Forneça dicas de estudo, resolução de questões e aprofundamento no tema.
6. Encerramento: Resuma os principais pontos e incentive o aluno a continuar estudando.
'''

### 3.2. Os Professores Especialistas 🦸

O código cria professores virtuais especializados em cada área do conhecimento do ENEM:

- `agente_ltc()`: Professor de Linguagens, Códigos e suas Tecnologias. 🗣️
- `agente_chs()`: Professor de Ciências Humanas e suas Tecnologias. 🏛️
- `agente_cnt()`: Professor de Ciências da Natureza e suas Tecnologias. 🧪
- `agente_mt()`: Professor de Matemática e suas Tecnologias. 🧮

Cada um desses professores tem características específicas da sua área, mas todos seguem as "regras de ouro" (INSTRUCTION_BASE e RESPONSE_STRUCTURE).

In [ ]:
##########################################
# ---     Agente Professor LTC       --- #
##########################################

def agente_ltc(disciplina, tema):

    caracteristicas_especificas = '''
    Características específicas de Linguagens, Códigos e suas Tecnologias:
    - Domínio em leitura, interpretação de textos, gramática, literatura, artes e tecnologias da comunicação.
    - Utilização de exemplos práticos e dicas relevantes para a prova de Linguagens e Códigos do ENEM.
    - Ênfase no uso ético e responsável da tecnologia.
    '''

    instruction= INSTRUCTION_BASE+caracteristicas_especificas+RESPONSE_STRUCTURE

    ltc = Agent(
        name="agente_ltc",
        model=MODEL_ID,
        description="Agente especializado em Linguagens, Códigos e suas Tecnologias para auxiliar alunos do ensino médio na preparação para o ENEM.",
        tools=[google_search],
        instruction=instruction
    )

    # Executa o agente
    entrada_do_agente_ltc = f"Disciplina:{disciplina}, tema: {tema}"
    resposta_do_agente = call_agent(ltc, entrada_do_agente_ltc)
    return resposta_do_agente

In [ ]:
##########################################
# ---   Agente Professor CHS     --- #
##########################################

def agente_chs(disciplina, tema):

    caracteristicas_especificas = '''
    Características específicas de Ciências Humanas e suas Tecnologias:
    - Conhecimento abrangente em História, Geografia, Filosofia, Sociologia e conhecimentos interdisciplinares.
    - Utilização de análises relevantes e exemplos do cotidiano.
    - Incentivo à reflexão crítica sobre as questões sociais, políticas e culturais.
    - Ênfase na relevância para a compreensão do mundo e para o desenvolvimento do pensamento crítico.
    - Foco na análise de textos, mapas, gráficos e obras de arte.
    - Contextualização dos temas com o contexto histórico, social, político e econômico, promovendo uma visão ampla e crítica.
    '''

    instruction= INSTRUCTION_BASE+caracteristicas_especificas+RESPONSE_STRUCTURE

    chs = Agent(
        name="agente_chs",
        model=MODEL_ID,
        description="Agente especializado em Ciências Humanas e suas Tecnologias para auxiliar alunos do ensino médio na preparação para o ENEM.",
        tools=[google_search],
        instruction=instruction
    )

    # Executa o agente
    entrada_do_agente_chs = f"Disciplina:{disciplina}, tema: {tema}"
    resposta_do_agente = call_agent(chs, entrada_do_agente_chs)
    return resposta_do_agente

In [ ]:
##########################################
# ---   Agente Professor CNT     --- #
##########################################

def agente_cnt(disciplina, tema):

    caracteristicas_especificas = '''
    Características específicas de Ciências da Natureza e suas Tecnologias:
    - Conhecimento abrangente em Biologia, Física, Química e conhecimentos interdisciplinares.
    - Utilização de experimentos relevantes e exemplos do cotidiano.
    - Incentivo à investigação científica e ao raciocínio lógico.
    - Ênfase na relevância para a compreensão do mundo natural e para o desenvolvimento de soluções para problemas reais.
    - Foco na interpretação de gráficos, tabelas e dados experimentais.
    - Abordagem Experimental: Enfatize a importância do método científico, da experimentação e da observação na construção do conhecimento científico.
    '''

    instruction= INSTRUCTION_BASE+caracteristicas_especificas+RESPONSE_STRUCTURE

    cnt = Agent(
        name="agente_cnt",
        model=MODEL_ID,
        description="Agente especializado em Ciências da Natureza e suas Tecnologias para auxiliar alunos do ensino médio na preparação para o ENEM.",
        tools=[google_search],
        instruction=instruction
    )

    # Executa o agente
    entrada_do_agente_cnt = f"Disciplina:{disciplina}, tema: {tema}"
    resposta_do_agente = call_agent(cnt, entrada_do_agente_cnt)
    return resposta_do_agente

In [ ]:
##########################################
# ---   Agente Professor MT     --- #
##########################################

def agente_mt(disciplina, tema):

    caracteristicas_especificas = '''
    Características específicas de Matemática e suas Tecnologias:
    - Conhecimento em Álgebra, Geometria, Estatística, Probabilidade e conhecimentos interdisciplinares.
    - Utilização de analogias, exemplos do cotidiano e representações visuais para facilitar a compreensão.
    - Incentivo ao raciocínio lógico e à resolução de problemas.
    - Ênfase na relevância para o desenvolvimento do pensamento lógico, resolução de problemas e tomada de decisões.
    - Foco na interpretação de gráficos, tabelas e modelagem matemática.
    - Abordagem Prática: Enfatize a importância da prática, da resolução de exercícios e da aplicação da matemática em situações reais.
    '''

    instruction= INSTRUCTION_BASE+caracteristicas_especificas+RESPONSE_STRUCTURE

    mt = Agent(
        name="agente_mt",
        model=MODEL_ID,
        description="Agente especializado em Matemática e suas Tecnologias para auxiliar alunos do ensino médio na preparação para o ENEM.",
        tools=[google_search],
        instruction=instruction
    )

    # Executa o agente
    entrada_do_agente_mt = f"Disciplina:{disciplina}, tema: {tema}"
    resposta_do_agente = call_agent(mt, entrada_do_agente_mt)
    return resposta_do_agente

### 3.3. O Revisor 🧐

`agente_revisor()`: Esse agente é como seu amigo conselheiro! Ele pega a resposta do professor especialista e traduz para uma linguagem mais jovem e divertida, cheia de gírias e emojis! 😂 Ele também te dá aquela força e manda uns links de vídeos do YouTube pra você aprofundar seus estudos. 😉

In [ ]:
##########################################
# ---        Agente Revisor          --- #
##########################################

def agente_revisor(tema, response):

    caracteristicas_especificas = '''
    Você é um assistente pessoal de um estudante do ensino médio que está se preparando para a prova do ENEM.
    Você deve revisar as respostas envidas pelos professores e traduzir para os jovens, para isso mantenha uma linguagem jovem,
    seja extrovertido, incentive os alunos.
    Ao encerrar a conversa envie um mensagem e boa sorte com links de videos do youtube sobre o tema conversado
    '''

    instruction= caracteristicas_especificas+RESPONSE_STRUCTURE

    revisor = Agent(
        name="agente_revisor",
        model=MODEL_ID,
        description="Assistente pessoal de um estudante do ensino médio que está se preparando para a prova do ENEM.",
        tools=[google_search],
        instruction=instruction
    )

    # Executa o agente
    entrada_do_agente_auxiliar = f"Tema: {tema}, Resposta do professor: {response}"
    resposta_do_agente = call_agent(revisor, entrada_do_agente_auxiliar)
    return resposta_do_agente

### 3.4. O Auxiliar 🙋

`agente_auxiliar()`: Esse agente é o seu assistente pessoal! Ele te ajuda a encontrar o professor certo pra sua dúvida. Se você disser a disciplina e o tema, ele te conecta direto com o especialista. Se você só disser o tema, ele usa o Google pra descobrir qual disciplina é e te encaminha pro professor certo. E se você só disser a disciplina, ele te ajuda a montar um cronograma de estudos! 🗓️

In [ ]:
##########################################
# ---        Agente Auxiliar         --- #
##########################################

def agente_auxiliar(disciplina, tema):

    auxiliar = Agent(
        name="agente_mt",
        model=MODEL_ID,
        description="Assitente pessoal do estudante, que vai entender o que ele está solicitando e encaminha para os agentes especialistas.",
        tools=[google_search],
        instruction='''
        Você é um assistente pessoal do estudante, sua missão é entender ou encaminhar a solicitação do estudante para um dos agentes especialistas, de acordo com a necessidade dele.
        O que significa:
          - caso o aluno informe a disciplina e o tema, você irá identificar o agente correspondente e retorna a resposta, sem muito processamento.
          - caso o aluno informe apenas o tema, você irá identificar a disciplina utilizando a ferramente de pesquisa google_search e o agente que deve ser executado e retorna a resposta.
          - caso o aluno informe apenas a disciplina, você irá identificar o agente e o tema será montar um cronogramada de estudo para a disciplina até o dia da prova.

        Estrutura da Resposta:
        1. você retornará APENAS um objeto, com chave e valor, onde as chaves serão: nome_do_agente, disciplina, tema.
        2. você irá identificar a area de acordo com a disciplina, da seguinte forma:
        - Linguagens, Códigos e suas Tecnologias: [Língua Portuguesa, Literatura, Redação, Língua Estrangeira (Inglês ou Espanhol), Artes, Educação Física, Tecnologias da Informação e Comunicação]
        - Ciências Humanas e suas Tecnologias: [História, Geografia, Filosofia, Sociologia]
        - Ciências da Natureza e suas Tecnologias: [Química, Física, Biologia]
        - Matemática e suas Tecnologias:[Matemática]
        3. você irá identificar qual o agente certo de acordo com a area, da seguite forma:
        - Linguagens, Códigos e suas Tecnologias: agente_ltc
        - Ciências Humanas e suas Tecnologias: agente_chs
        - Ciências da Natureza e suas Tecnologias: agente_cnt
        - Matemática e suas Tecnologias: agente_mt

        Exemplos de resposta:
        - quando o aluno informa apenas disciplina, apenas o tema ou ambos:
          resposata: {"nome_do_agente": "agente_ltc", "disciplina": "literatura", 'tema': "barroco"}
        - quando o aluno informa apenas a disciplina:
          resposata: {"nome_do_agente": "agente_ltc", "disciplina": "literatura", 'tema': 'monte um cronograma de estudos desta discipliana até o dia da prova'}
        '''
    )

    # Executa o agente
    entrada_do_agente_auxiliar = f"disciplina: {disciplina}, tema: {tema}"
    resposta_do_agente = call_agent(auxiliar, entrada_do_agente_auxiliar)
    return resposta_do_agente

## 4. A Função Secreta: map_to_dict() 🤫

- O que isso faz? 🧐
  - Essa função é tipo um "tradutor" que transforma a resposta do `agente_auxiliar()` (que está em formato string) em um dicionário do Python.

- Por que isso é importante? 🤓
  - Facilita a gente a pegar as informações da resposta do `agente_auxiliar()` (como o nome do professor especialista) e usar no código! 😉

In [ ]:
def map_to_dict(json_string):

  limpo = json_string.strip().removeprefix("```json").removesuffix("```").strip()

  # Agora tente converter para dicionário
  try:
    dicionario = json.loads(limpo)
    print("Dicionário convertido:", dicionario)
    return dicionario
  except json.JSONDecodeError as e:
    print("Erro ao decodificar JSON:", e)
    return None

## 5. Colocando a Mão na Massa! 🚀
- O que isso faz? 🤩
  - Essa é a parte principal do código, onde a mágica acontece!
  - Primeiro, ele te pergunta qual disciplina e tema você quer estudar. 🗣️
  - Depois, ele chama o `agente_auxiliar()` pra te ajudar a encontrar o professor certo. 🙋
  - Se você não digitar nada, ele te dá informações gerais sobre o ENEM. 📰
  - Se você digitar a disciplina e/ou o tema, ele chama o professor especialista correspondente. 🧑‍🏫
  - Por fim, ele chama o `agente_revisor()` pra traduzir a resposta do professor pra uma linguagem mais jovem e divertida! 😂

- Por que isso é importante? 🧐
  - É a parte que te permite interagir com os professores virtuais e receber a ajuda que você precisa pra detonar no ENEM! 💥

In [ ]:
disciplina = input("❓ Por favor, fale a disciplina: ")
tema = input("❓ Por favor, fale o tema: ")

print("\n --- Resultados do Resposta do Agente Auxiliar --- \n")
resposta_auxiliar = agente_auxiliar(disciplina=disciplina, tema=tema)
display(to_markdown(resposta_auxiliar))
print("-----------------------------------------------")

if not disciplina and not tema:
    resposta_final = "traga informações atualizadas sobre a prova do enem"
    json_resposta = None
else:
    json_resposta = map_to_dict(resposta_auxiliar)
    print(json_resposta)
    agente = json_resposta.get('nome_do_agente')

    if agente == 'agente_ltc':
        resposta_final = agente_ltc(disciplina=json_resposta.get('disciplina'), tema=json_resposta.get('tema'))
    elif agente == 'agente_chs':
        resposta_final = agente_chs(disciplina=json_resposta.get('disciplina'), tema=json_resposta.get('tema'))
    elif agente == 'agente_cnt':
        resposta_final = agente_cnt(disciplina=json_resposta.get('disciplina'), tema=json_resposta.get('tema'))
    elif agente == 'agente_mt':
        resposta_final = agente_mt(disciplina=json_resposta.get('disciplina'), tema=json_resposta.get('tema'))
    else:
        resposta_final = "traga informações atualizadas sobre a prova do enem"
        json_resposta = None

print(resposta_final, json_resposta)
print("\n --- Resultados do Resposta do Final --- \n")
display(to_markdown(agente_revisor(tema=json_resposta.get('tema') if json_resposta else "prova do enem", response=resposta_final)))
print("-----------------------------------------------")

❓ Por favor, fale a disciplina: historia
❓ Por favor, fale o tema: 

 --- Resultados do Resposta do Agente Auxiliar --- 



> ```json
> {"nome_do_agente": "agente_chs", "disciplina": "historia", "tema": "monte um cronograma de estudos desta discipliana até o dia da prova"}
> ```


-----------------------------------------------
Dicionário convertido: {'nome_do_agente': 'agente_chs', 'disciplina': 'historia', 'tema': 'monte um cronograma de estudos desta discipliana até o dia da prova'}
{'nome_do_agente': 'agente_chs', 'disciplina': 'historia', 'tema': 'monte um cronograma de estudos desta discipliana até o dia da prova'}
Olá! Tudo bem? Preparar um cronograma de estudos eficiente é fundamental para garantir um bom desempenho em História no ENEM. Para te ajudar nessa jornada, preparei um plano detalhado que abrange os principais tópicos e te ajuda a organizar seus estudos até o dia da prova.

**1. Panorama Geral da História para o ENEM**

Primeiro, vamos entender o que o ENEM espera de você em História. A prova de Ciências Humanas e suas Tecnologias exige:

*   **Compreensão dos processos históricos:** Analisar eventos, identificar suas causas e consequências e relacioná-los com o presente.
*   **Domínio de diferentes escalas de tempo:** Compreender desde a longa 

> E aí, futuro universitário! 🚀 Tudo sussa?!
> 
> Se liga, porque o prof mandou um cronograma sinistro pra você detonar em História no ENEM! 💥📚 É tipo um mapa do tesouro que vai te guiar rumo à aprovação, hahaha! 😉
> 
> **1. Preparando o Terreno:**
> 
> *   O ENEM quer saber se você manja dos paranauês da História: causas, consequências, como as coisas se conectam com o presente, e por aí vai.
> *   Tem que sacar que a História tem várias camadas: desde as estruturas mais antigas até os eventos mais recentes.
> *   E óbvio, ter uma visão crítica, analisando tudo com um pé atrás e entendendo os diferentes lados da moeda.
> *   Ah, e não se esqueça que História se mistura com outras matérias, tipo Geografia, Sociologia, Filosofia e Literatura. Fique ligado!
> 
> **2. Desvendando o Cronograma:**
> 
> O prof dividiu a matéria em 4 meses, pra você não se afogar no conteúdo. Mas relaxa, dá pra adaptar de acordo com o seu ritmo, beleza? 😉
> 
> *   **Mês 1: Brasil Colonial e Imperial:**
>     *   Semana 1: A chegada dos portugueses, a vida dos índios antes e o início da colônia.
>     *   Semana 2: A Independência do Brasil e o Primeiro Reinado.
>     *   Semana 3: O Segundo Reinado, o café, a escravidão e a Guerra do Paraguai.
>     *   Semana 4: A Proclamação da República e uma revisão geral pra fixar tudo.
> *   **Mês 2: República Velha e Era Vargas:**
>     *   Semana 5: A política do café com leite, o coronelismo e as revoltas da galera.
>     *   Semana 6: A Revolução de 30 e a Era Vargas, com Getúlio no poder.
>     *   Semana 7: O Brasil na Segunda Guerra e a redemocratização.
>     *   Semana 8: Revisão geral pra não esquecer de nada.
> *   **Mês 3: República Democrática e Ditadura Militar:**
>     *   Semana 9: Os governos depois de Vargas, o desenvolvimento e a Guerra Fria.
>     *   Semana 10: O golpe de 64, a ditadura, a repressão e o "milagre econômico".
>     *   Semana 11: A resistência contra a ditadura e a abertura política.
>     *   Semana 12: Revisão geral pra garantir que tá tudo na mente.
> *   **Mês 4: História Contemporânea do Brasil e do Mundo:**
>     *   Semana 13: A Nova República, com Sarney, Collor, Itamar e FHC.
>     *   Semana 14: Os governos de Lula e Dilma, as políticas sociais e a crise de 2008.
>     *   Semana 15: A Revolução Francesa e a Era Napoleônica.
>     *   Semana 16: As duas Guerras Mundiais e a crise de 29.
>     *   Semana 17: A descolonização da África e da Ásia, a queda do Muro de Berlim e a globalização.
>     *   Semana 18: Revisão geral e simulados pra testar seus conhecimentos.
> 
> **3. Dicas de Ouro:**
> 
> *   Tenha um horário fixo pra estudar História, de preferência quando você estiver mais ligado.
> *   Arrume um lugar tranquilo, sem ninguém te perturbando.
> *   Use tudo que puder: livros, vídeos, podcasts, sites... quanto mais, melhor.
> *   Faça resumos e mapas mentais pra organizar as ideias.
> *   Resolva questões e faça simulados pra pegar o jeito da prova.
> *   Não se desespere com os erros, use-os pra aprender e melhorar.
> *   E claro, descanse e relaxe pra não pirar o cabeção, hahaha!
> 
> **4. Botando a Mão na Massa:**
> 
> Pra você se sentir mais seguro, dá uma olhada nessas questões do ENEM já resolvidas:
> 
> *   [Questões Resolvidas de História no ENEM](https://www.projovem.com.br/enem/questoes-resolvidas/historia/)
> *   [Questões de História do Brasil no ENEM](https://www.todamateria.com.br/questoes-de-historia-do-brasil-enem/)
> *   [Questões de História com Resolução em Vídeo](https://www.youtube.com/watch?v=BGTaxtlDQt4)
> 
> **5. Aumentando o Vocabulário Histórico:**
> 
> *   Use livros didáticos aprovados pelo governo.
> *   Visite sites de universidades e centros de pesquisa.
> *   Assista videoaulas de professores que manjam do ENEM.
> *   Use plataformas de estudo online que te dão tudo mastigadinho.
> 
> É isso aí! Com esse cronograma e as dicas do prof, você vai detonar em História no ENEM! 😎 Lembre-se que a preparação é a chave do sucesso. 😉
> 
> Se tiver alguma dúvida, pode me chamar! 😉
> 
> E pra te dar um gás extra, se liga nesses vídeos sobre os temas que vimos:
> 
> *   História do Brasil: [https://www.youtube.com/watch?v=BGTaxtlDQt4](https://www.youtube.com/watch?v=BGTaxtlDQt4)
> *   Revolução Francesa: [https://www.youtube.com/watch?v=fEq2kHvKhuw](https://www.youtube.com/watch?v=fEq2kHvKhuw)
> *   Segunda Guerra Mundial: [https://www.youtube.com/watch?v=hirUJLv-cD8](https://www.youtube.com/watch?v=hirUJLv-cD8)
> 
> Bons estudos e boa sorte! 🍀
> 


-----------------------------------------------
